In [5]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
conda install -c anaconda lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.3.0-py36hefd8a0e_0 anaconda


lxml-4.3.0           | 1.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c anaconda lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.3.0-py36hefd8a0e_0 anaconda


lxml-4.3.0           | 1.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [13]:
import requests
import pandas as pd
import lxml.html as lh

In [14]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [16]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [17]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [18]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [19]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[287, 287, 287]

In [20]:
#Creating the pandas data frame

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [23]:
#Rearranging and renaming the columns

df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned\n,M1A,Not assigned
1,Not assigned\n,M2A,Not assigned
2,Parkwoods\n,M3A,North York
3,Victoria Village\n,M4A,North York
4,Harbourfront\n,M5A,Downtown Toronto


In [24]:
#Cleaning the messy string in the Borough column

df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


In [25]:
#Dropping all cells with a borough that is Not assigned

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto
5,Lawrence Heights,M6A,North York
6,Lawrence Manor,M6A,North York
7,Not assigned,M7A,Queen's Park
8,Not assigned,M8A,Not assigned
9,Queen's Park,M9A,Downtown Toronto


In [26]:
#Combining Neighbourhoods based on similar Postcode and Borough

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke
5,Bathurst Manor,M3H,North York
6,Bathurst Quay,M5V,Downtown Toronto
7,Bayview Village,M2K,North York
8,Beaumond Heights,M9V,Etobicoke
9,Bedford Park,M5M,North York


In [27]:
#Remove any space in the start of the string

df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [28]:
df.head()

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke


In [29]:
#Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [30]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood


In [31]:
# Check the shape of the data frame
df.shape

(287, 3)

In [32]:
df.to_csv(r'df_can.csv')